In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import dask
import intake
import fsspec
from collections import defaultdict
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
from xmip.utils import google_cmip_col
from xmip.preprocessing import combined_preprocessing
from xmip.postprocessing import merge_variables

/tmp/ipykernel_810/3693849798.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!


In [2]:
def regrid_to_era5(ds,era5_grid):
    """wrapper around xesmf regridding"""
    regridder = xe.Regridder(ds,era5_grid,'bilinear',ignore_degenerate=True)
    
    return regridder(ds)


In [3]:
my_models = ['BCC-CSM2-MR','CESM2''CESM2-WACCM','CMCC-ESM2','CMCC-CM2-SR5','EC-Earth3',
                'GFDL-CM4','GFDL-ESM4','HadGEM3-GC31-MM','MIROC6','MPI-ESM1-2-HR','MRI-ESM2-0',
                'NorESM2-MM','TaiESM1']

col = google_cmip_col()
experiment_id='ssp585'
source_id = my_models
kwargs = {
    'zarr_kwargs':{
        'consolidated':True,
        'use_cftime':True
    },
    'aggregate':False
}

cat_data = col.search(
    source_id=source_id,
    experiment_id=experiment_id,
    table_id='day',
    variable_id=['psl','sfcWind']
)
ddict = cat_data.to_dataset_dict(**kwargs)

#NB: I'm not applying any preprocessing/renaming here at the moment because the regridding seems to work fine. The atmospheric fields are typically a bit more straightforward to handle.

/tmp/ipykernel_810/3627879381.py:22: DeprecationWarning: cdf_kwargs and zarr_kwargs are deprecated and will be removed in a future version. Please use xarray_open_kwargs instead.
  ddict = cat_data.to_dataset_dict(**kwargs)



--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.member_id.table_id.variable_id.grid_label.zstore.dcpp_init_year.version'


In [4]:
ddict_merged = merge_variables(ddict) #skips model realizations for which simulation time differs (see warnings), need to fix this
#list(ddict_merged.keys())

/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/postprocessing.py:157: UserWarning: ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r121i1p1f1.day.gr.none.psl failed to combine with :cannot align objects with join='exact' where index/labels/sizes are not equal along these coordinates (dimensions): 'time' ('time',)
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/postprocessing.py:157: UserWarning: ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r103i1p1f1.day.gr.none.psl failed to combine with :cannot align objects with join='exact' where index/labels/sizes are not equal along these coordinates (dimensions): 'time' ('time',)
  warnings.warn(f"{cmip6_dataset_id(ds)} failed to combine with :{e}")
/srv/conda/envs/notebook/lib/python3.10/site-packages/xmip/postprocessing.py:157: UserWarning: ScenarioMIP.EC-Earth-Consortium.EC-Earth3.ssp585.r120i1p1f1.day.gr.none.psl failed to combine with :cannot 

In [5]:
reqVars = ['sfcWind','psl'] #need both these variables
ddict_filtered = {k: v for k, v in ddict_merged.items() if set(reqVars).issubset(list(ddict_merged[k].variables))} #drop datasets which don't have both

In [6]:
def concat_realizations_most_common_ipf(ds_list):
    '''custom function that concatenates only the realizations of the most common 'ipf' combination,
    takes the first sorted 'ipf' if multiple 'ipf' are equally common'''
    
    member_ids = [ds.member_id.data[0] for ds in ds_list]
    
    member_ids.sort() #often i1 is the baseline?

    ipf_ids = [s[s.find('i'):] for s in member_ids] #separate 'ipf' from 'r'
    from collections import Counter

    most_common_ipf = Counter(ipf_ids).most_common()[0][0]

    # find unique members and decide which values of 'ipf' give the most members/variants?
    # pick only the matching datasets from the list
    ds_pick = [ds for ds in ds_list if (most_common_ipf in ds.member_id.data[0])]

    return xr.concat(ds_pick, dim='member_id')

from xmip.postprocessing import combine_datasets

ddict_concat = combine_datasets(
    ddict_filtered,
    concat_realizations_most_common_ipf,
    match_attrs=['source_id', 'grid_label', 'experiment_id', 'table_id']
)
#NB: This leaves multiple datasets for the same model with different grid labels. 
# That is not straightforward to filter out if querying multiple experiments. 
# Since this occurs only for a few models it is probably OK to just save these and remove them later.


In [7]:
mlrcoefs = xr.open_dataset('/home/jovyan/CMIP6cf/gssr_coefs_1degRes_forcing.nc') #contains coordinates of and MLR coefficients around TGs

era5_grid = xr.Dataset( #the ERA5 grid used to derive the MLR coefficients
        {
            "longitude": (["longitude"], np.arange(-40,30,1)+1/2, {"units": "degrees_east"}),
            "latitude": (["latitude"], np.arange(70,10,-1)-1/2, {"units": "degrees_north"}),
        }
    )

#get coordinates of n x n degree grids around each tide gauge
num_degr = 2
lat_ranges = np.zeros((len(mlrcoefs.tg),2))
lon_ranges = np.zeros((len(mlrcoefs.tg),2))

for t,tg in enumerate(mlrcoefs.tg.values):
    lat_ranges[t,:] = era5_grid.latitude[((era5_grid.latitude>=(mlrcoefs.sel(tg=tg).lat-num_degr/2)) & (era5_grid.latitude<=(mlrcoefs.sel(tg=tg).lat+num_degr/2)))][0:2]
    lon_ranges[t,:] = era5_grid.longitude[((era5_grid.longitude>=(mlrcoefs.sel(tg=tg).lon-num_degr/2)) & (era5_grid.longitude<=(mlrcoefs.sel(tg=tg).lon+num_degr/2)))][0:2]

#create da's to index the CMIP6 files with:
lons_da = xr.DataArray(lon_ranges,dims=['tg','lon_around_tg'],coords={'tg':mlrcoefs.tg,'lon_around_tg':[0,1]})
lats_da = xr.DataArray(lat_ranges,dims=['tg','lat_around_tg'],coords={'tg':mlrcoefs.tg,'lat_around_tg':[0,1]})

In [12]:
ddict_subsetted = ddict_concat #copy dictionary with concatenated realizations
for key in ddict_concat:
    ds = ddict_concat[key]

    #change longitude coordinates to -180 -> 180 (avoids getting NaNs at the 0-meridian)
    lon_coord = list(k for k in ds.dims if 'lon' in k)[0] #find lon/lat coordinate names
    ds.coords[lon_coord] = ((ds.coords[lon_coord] + 180) % 360) - 180 #wrap around 0
    ds = ds.reindex({ lon_coord : np.sort(ds[lon_coord])})
    
    regridded_ds = regrid_to_era5(ds,era5_grid) #regrid to the ERA5 grid bilinearly
    ds_around_tgs = regridded_ds.sel(latitude=lats_da,longitude=lons_da) #subset at n x n degree grids around TGs
    ddict_subsetted[key] = ds_around_tgs
    #ds_around_tgs.to_netcdf('test.nc',mode='w') #storing the subsetted output

In [10]:
ddict_subsetted.keys()

dict_keys(['HadGEM3-GC31-MM.gn.ssp585.day', 'GFDL-CM4.gr2.ssp585.day', 'EC-Earth3.gr.ssp585.day', 'GFDL-CM4.gr1.ssp585.day', 'CMCC-ESM2.gn.ssp585.day', 'CMCC-CM2-SR5.gn.ssp585.day', 'GFDL-ESM4.gr1.ssp585.day', 'MPI-ESM1-2-HR.gn.ssp585.day', 'BCC-CSM2-MR.gn.ssp585.day', 'MRI-ESM2-0.gn.ssp585.day', 'MIROC6.gn.ssp585.day', 'NorESM2-MM.gn.ssp585.day', 'TaiESM1.gn.ssp585.day'])

In [11]:
ddict_subsetted['HadGEM3-GC31-MM.gn.ssp585.day'].to_netcdf('test.nc',mode='w')

0it [00:00, ?it/s]